# Examen d'ocvx 2

## Section 1

### Question 1

In [4]:
import warnings
import math
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

In [5]:
class test_function():
    
    def __init__(self, name, value, dim, nb_params, convex=None, grad=None, hess=None, HCN=None, constraints = None):
        self.name = name
        self.value = value
        self.dim = dim
        self.nb_params = nb_params
        self.convex = convex
        self.grad = grad
        self.hess = hess
        self.HCN = HCN
        self.constraints = constraints

In [261]:
mini_1_d = {
    "name": "mini_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : True,
    "value" : (lambda x: np.square([x])-x+2),
    "grad"  : (lambda x: np.array([2*x - 1])),
    "hess"  : (lambda x: np.array([2])),
    "HCN"   : 1,
    "constraints": []
}

mini_1 = test_function(**mini_1_d)

In [262]:
mini_1.hess(np.array(3))

array([2])

In [194]:
maxi_1_d = {
    "name": "mini_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : True,
    "value" : (lambda x: -1*np.square([x])+2*x+1),
    "grad"  : (lambda x: np.array([-2*x + 2])),
    "hess"  : (lambda x: np.array([-2])),
    "HCN"   : 1,
    "constraints": []
}

maxi_1 = test_function(**maxi_1_d)

In [199]:
maxi_1.hess(np.array(3))

array([-2])

In [8]:
#C'est la fonction de Bashar, faut changer un peu

func_3_d = {
    "name": "function_3",
    "dim" : 2,
    "nb_params" : 1,
    "convex" : True ,
    "value" : (lambda x, gamma: x[:, 0]**2 + gamma*x[:, 1]**2),
    "grad"  : (lambda x, gamma: np.array([2*x[:, 0], 2*gamma*x[:, 1]]).reshape(1, -1)),
    "hess"  : (lambda x, gamma: np.diag([2, 2*gamma])),
    "HCN"   : (lambda x, gamma: gamma),
    "constraints" : [],
}

func_3 = test_function(**func_3_d)

In [9]:
exp_1_d = {
    "name": "exp_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : True,
    "value" : (lambda x: math.exp(x)+math.exp(-x)-1),
    "grad"  : (lambda x: math.exp(x)-math.exp(-x)),
    "hess"  : (lambda x: math.exp(-x)+math.exp(x)),
    "HCN"   : 1, #jsp
    "constraints": []
}

exp_1 = test_function(**exp_1_d)

In [57]:
test_list = [mini_1, maxi_1, exp_1]

### Question 2

In [264]:
def backtracking(f, x, dir_x, alpha=0.01, beta=0.8):
    
    x = x.reshape(1, -1)
    t = 1

    while f.value(x+t*dir_x) > f.value(x) + alpha*t*np.transpose(f.grad(x))*dir_x:
        t = beta*t

    return t

In [270]:
def newton(f, x, epsilon=0.0001):
    
    x = x.reshape(1, -1)
    if (len(f.hess(x).shape) == 1):
        hess_inv = 1/f.hess(x)
    else:
        hess_inv = np.linalg.inv(f.hess(x))

    x_dir = -1*hess_inv*f.grad(x)
    l = abs(-1*np.transpose(f.grad(x))*x_dir)
    
    while abs(l/2) > epsilon:
        
        x_dir = -1*hess_inv*f.grad(x)
        l = -1*np.transpose(f.grad(x))*x_dir
        #t = backtracking(f, x, x_dir)
        x = (x+x_dir).reshape(1, -1)#+t*x_dir).reshape(1, -1)
        
    return x

In [271]:
newton(maxi_1, np.array(2.5))

array([[1.]])

### Question 3

In [83]:
# code de Bashar pour la descente de gradient

def partial(f, x, i=0, dx=1e-6):
    """Computes i-th partial derivative of f at point x.
    
    Args:
        f: objective function.
        x: point at which partial derivative is computed.
        i: coordinate along which derivative is computed.
        dx: slack for finite difference.
        
    Output:
        (float)

    """
    x = x.reshape(1, -1)
    h = np.zeros(x.shape)
    h[0, i] = dx
    return (f(x + h) - f(x - h)) / (2*dx)

def gradient(f, x, dx=1e-6):
    """Computes gradient of f at point x.
    
    Args:
        f: objective function.
        x: point at which gradient is computed.
        dx: slack for finite difference of partial derivatives.
        
    Output:
        (ndarray) of size domain of f.
        
    """
    x = x.reshape(1, -1)
    dim = x.shape[1]
    return np.array([partial(f, x, i, dx) for i in range(dim)]).reshape(1, -1)